In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1667, -0.0160,  0.1098, -0.0031,  0.3437,  0.0358,  0.2356,  0.1512,
         -0.0883, -0.2813],
        [-0.1805,  0.1097,  0.1156, -0.0256,  0.1931, -0.0906,  0.2036,  0.1237,
         -0.0111, -0.2872]], grad_fn=<AddmmBackward>)

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X)

tensor([[-0.0227, -0.0892,  0.1936,  0.2009, -0.0352, -0.0845, -0.0455,  0.1223,
         -0.2091,  0.3325],
        [ 0.0274, -0.1942,  0.2438,  0.2046, -0.2316, -0.0767,  0.0493,  0.0318,
         -0.1483,  0.3009]], grad_fn=<AddmmBackward>)

In [9]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
            
 
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [10]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X) 

tensor([[ 0.3331,  0.1622,  0.0344,  0.0973,  0.0008,  0.0673, -0.3289,  0.0305,
         -0.2681, -0.2594],
        [ 0.1483,  0.0526, -0.0689, -0.0038,  0.0690,  0.0782, -0.3594,  0.0553,
         -0.2690, -0.2525]], grad_fn=<AddmmBackward>)

In [11]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [12]:
net = FixedHiddenMLP()
net(X)

tensor(0.2565, grad_fn=<SumBackward0>)

In [14]:
class NeseMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
 
    def forward(self, X):
        return self.linear(self.net(X))
    
 
chimera = nn.Sequential(NeseMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1718, grad_fn=<SumBackward0>)